In [11]:
#Import libraries:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn import  metrics
from sklearn.model_selection import GridSearchCV

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [12]:
train = pd.read_csv('train_modified.csv')
target = 'Disbursed'
IDcol = 'ID'

In [14]:
train.head()

,ID,Gender,City,Monthly_Income,DOB,Lead_Creation_Date,Loan_Amount_Applied,Loan_Tenure_Applied,Existing_EMI,Employer_Name,...,Interest_Rate,Processing_Fee,EMI_Loan_Submitted,Filled_Form,Device_Type,Var2,Source,Var4,LoggedIn,Disbursed
0,ID000002C20,Female,Delhi,20000,23-May-78,15-May-15,300000.0,5.0,0.0,CYBOSOL,...,NaN,NaN,NaN,N,Web-browser,G,S122,1,0,0
1,ID000004E40,Male,Mumbai,35000,07-Oct-85,04-May-15,200000.0,2.0,0.0,TATA CONSULTANCY SERVICES LTD (TCS),...,13.25,NaN,6762.9,N,Web-browser,G,S122,3,0,0
2,ID000007H20,Male,Panchkula,22500,10-Oct-81,19-May-15,600000.0,4.0,0.0,ALCHEMIST HOSPITALS LTD,...,NaN,NaN,NaN,N,Web-browser,B,S143,1,0,0
3,ID000008I30,Male,Saharsa,35000,30-Nov-87,09-May-15,1000000.0,5.0,0.0,BIHAR GOVERNMENT,...,NaN,NaN,NaN,N,Web-browser,B,S143,3,0,0
4,ID000009J40,Male,Bengaluru,100000,17-Feb-84,20-May-15,500000.0,2.0,25000.0,GLOBAL EDGE SOFTWARE,...,NaN,NaN,NaN,N,Web-browser,B,S134,3,1,0


In [15]:
train.columns

Index(['ID', 'Gender', 'City', 'Monthly_Income', 'DOB', 'Lead_Creation_Date',
       'Loan_Amount_Applied', 'Loan_Tenure_Applied', 'Existing_EMI',
       'Employer_Name', 'Salary_Account', 'Mobile_Verified', 'Var5', 'Var1',
       'Loan_Amount_Submitted', 'Loan_Tenure_Submitted', 'Interest_Rate',
       'Processing_Fee', 'EMI_Loan_Submitted', 'Filled_Form', 'Device_Type',
       'Var2', 'Source', 'Var4', 'LoggedIn', 'Disbursed'],
      dtype='object')

In [16]:
train.size

2262520

In [18]:
train.nunique()

ID                       87020
Gender                       2
City                       697
Monthly_Income            5825
DOB                      11345
Lead_Creation_Date          92
Loan_Amount_Applied        277
Loan_Tenure_Applied         11
Existing_EMI              3753
Employer_Name            43567
Salary_Account              57
Mobile_Verified              2
Var5                        19
Var1                        19
Loan_Amount_Submitted      203
Loan_Tenure_Submitted        6
Interest_Rate               73
Processing_Fee             571
EMI_Loan_Submitted        4530
Filled_Form                  2
Device_Type                  2
Var2                         7
Source                      30
Var4                         8
LoggedIn                     2
Disbursed                    2
dtype: int64

In [22]:
train.isnull().sum()

ID                           0
Gender                       0
City                      1003
Monthly_Income               0
DOB                          0
Lead_Creation_Date           0
Loan_Amount_Applied         71
Loan_Tenure_Applied         71
Existing_EMI                71
Employer_Name               71
Salary_Account           11764
Mobile_Verified              0
Var5                         0
Var1                         0
Loan_Amount_Submitted    34613
Loan_Tenure_Submitted    34613
Interest_Rate            59294
Processing_Fee           59600
EMI_Loan_Submitted       59294
Filled_Form                  0
Device_Type                  0
Var2                         0
Source                       0
Var4                         0
LoggedIn                     0
Disbursed                    0
dtype: int64

In [23]:
print(train.apply(lambda col: col.nunique()))

ID                       87020
Gender                       2
City                       697
Monthly_Income            5825
DOB                      11345
Lead_Creation_Date          92
Loan_Amount_Applied        277
Loan_Tenure_Applied         11
Existing_EMI              3753
Employer_Name            43567
Salary_Account              57
Mobile_Verified              2
Var5                        19
Var1                        19
Loan_Amount_Submitted      203
Loan_Tenure_Submitted        6
Interest_Rate               73
Processing_Fee             571
EMI_Loan_Submitted        4530
Filled_Form                  2
Device_Type                  2
Var2                         7
Source                      30
Var4                         8
LoggedIn                     2
Disbursed                    2
dtype: int64


In [47]:
print(train.apply(lambda col: col.unique()))

ID                       [ID000002C20, ID000004E40, ID000007H20, ID0000...
Gender                                                              [0, 1]
City                     [Delhi, Mumbai, Panchkula, Saharsa, Bengaluru,...
Monthly_Income           [20000, 35000, 22500, 100000, 45000, 70000, 75...
DOB                      [23-May-78, 07-Oct-85, 10-Oct-81, 30-Nov-87, 1...
Lead_Creation_Date       [15-May-15, 04-May-15, 19-May-15, 09-May-15, 2...
Loan_Amount_Applied      [300000.0, 200000.0, 600000.0, 1000000.0, 5000...
Loan_Tenure_Applied      [5.0, 2.0, 4.0, 0.0, 3.0, 1.0, 10.0, 7.0, nan,...
Existing_EMI             [0.0, 25000.0, 15000.0, 2597.0, 4600.0, 1200.0...
Employer_Name            [CYBOSOL, TATA CONSULTANCY SERVICES LTD (TCS),...
Salary_Account           [HDFC Bank, ICICI Bank, State Bank of India, H...
Mobile_Verified                                                     [N, Y]
Var5                     [0, 13, 10, 17, 3, 11, 14, 12, 5, 8, 1, 4, 6, ...
Var1                     

In [73]:
train.columns

Index(['ID', 'Gender', 'Monthly_Income', 'DOB', 'Lead_Creation_Date',
       'Loan_Amount_Applied', 'Loan_Tenure_Applied', 'Existing_EMI',
       'Employer_Name', 'Salary_Account', 'Mobile_Verified', 'Var5', 'Var1',
       'Loan_Amount_Submitted', 'Loan_Tenure_Submitted', 'Interest_Rate',
       'Processing_Fee', 'Filled_Form', 'Device_Type', 'Var2', 'Source',
       'Var4', 'LoggedIn', 'Disbursed'],
      dtype='object')

In [53]:
train.drop(["EMI_Loan_Submitted"], axis = 1, inplace = True)

In [72]:
train.drop(["City"], axis = 1, inplace = True)

In [110]:
train['DOB1'] = pd.to_datetime(train['DOB'])

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1-01-04 00:00:00

In [107]:
train.dtypes

ID                               object
Gender                            int32
Monthly_Income                    int64
DOB                              object
Lead_Creation_Date               object
Loan_Amount_Applied             float64
Loan_Tenure_Applied             float64
Existing_EMI                    float64
Employer_Name                    object
Salary_Account                   object
Mobile_Verified                  object
Var5                              int64
Var1                             object
Loan_Amount_Submitted           float64
Loan_Tenure_Submitted           float64
Interest_Rate                   float64
Processing_Fee                  float64
Filled_Form                      object
Device_Type                      object
Var2                             object
Source                           object
Var4                              int64
LoggedIn                          int64
Disbursed                         int64
DOB1                     datetime64[ns]


In [39]:
pd.DataFrame(train.Gender.unique())

,0
0,Female
1,Male


In [41]:
# Import label encoder
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder

In [43]:
# label_encoder object knows how to understand word labels.
label_encoder = LabelEncoder()
 
# Encode labels in column 'species'.
train['Gender'] = label_encoder.fit_transform(train['Gender'])
 
train['Gender'].unique()

array([0, 1])

In [44]:
pd.DataFrame(train.Gender.unique())

,0
0,0
1,1


In [30]:
pd.DataFrame(train.City.unique())

,0
0,Delhi
1,Mumbai
2,Panchkula
3,Saharsa
4,Bengaluru
...,...
693,Tamenglong
694,DHORAJI
695,Siruguppa
696,Lakshadweep


In [45]:
def modelfit(alg, dtrain, predictors, performCV = True, printFeatureImportance = True, cv_folds = 5):
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Disbursed'])
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
    
    #Perform cross-validation:
    if performCV:
        cv_score = cross_validation.cross_val_score(alg, dtrain[predictors],
                                                    dtrain['Disbursed'],
                                                    cv = cv_folds, scoring='roc_auc')
    
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain['Disbursed'].values, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob))
    
    if performCV:
        print ("CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score),
                                                                                 np.std(cv_score),
                                                                                 np.min(cv_score),
                                                                                 np.max(cv_score)))
    
    #Print Feature Importance:
    if printFeatureImportance:
        feat_imp = pd.Series(alg.feature_importances_, predictors).sort_values(ascending=False)
        feat_imp.plot(kind='bar', title='Feature Importances')
        plt.ylabel('Feature Importance Score')

In [46]:
#Choose all predictors except target & IDcols
predictors = [x for x in train.columns if x not in [target, IDcol]]
gbm0 = GradientBoostingClassifier(random_state=10)
modelfit(gbm0, train, predictors)

ValueError: could not convert string to float: 'Delhi'